# Metric Design

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
#reading the cleaned master data 
master_data = pd.read_csv("Master Data.csv")

In [3]:
#glancing
master_data.head()

,hospital_pk,collection_week,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,inpatient_beds_used_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,...,previous_day_admission_adult_covid_confirmed_7_day_avg,previous_day_admission_pediatric_covid_confirmed_7_day_avg,previous_day_admission_adult_covid_suspected_7_day_avg,previous_day_admission_pediatric_covid_suspected_7_day_avg,all_pediatric_inpatient_bed_occupied_7_day_avg,all_pediatric_inpatient_beds_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_avg,total_staffed_pediatric_icu_beds_7_day_avg,state_full
0,330219,2020-12-04,4011,4011,3626,3297,3297,391,391,390,...,5.571429,0.0,0.285714,0.000000,0.0,0.0,0.0,0.0,0.0,New York
1,351312,2022-07-01,0,0,70,24,24,2,2,2,...,0.285714,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,North Dakota
2,452083,2020-09-25,343,343,343,335,335,145,145,145,...,1.571429,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,Texas
3,340114,2020-11-27,3947,3331,3331,2595,2588,375,375,319,...,5.000000,0.0,13.571429,0.285714,0.0,0.0,0.0,0.0,0.0,North Carolina
4,330406,2021-06-18,805,749,749,612,605,0,0,0,...,0.000000,0.0,0.000000,0.000000,1.0,8.0,0.0,0.0,0.0,New York


# Formulas:


### Calculating total confirmed covid beds occupied over total inpatient beds
inpatient_confirmed = inpatient_beds_used_covid_7_day_avg / inpatient_beds_7_day_avg

### Calculating total confirmed covid beds occupied over total ICU beds
icu_confirmed = (staffed_icu_adult_patients_confirmed_covid_7_day_avg + staffed_icu_pediatric_patients_confirmed_covid_7_day_avg) / total_icu_beds_7_day_avg

### Calcualting fully vaccinated staff over total staff
staff_vaccinated = total_personnel_covid_vaccinated_doses_all_7_day/( total_personnel_covid_vaccinated_doses_none_7_day + total_personnel_covid_vaccinated_doses_one_7_day + total_personnel_covid_vaccinated_doses_all_7_day)

### Calculating the hospital burden ranking based on initial weights assigned on the basis of domain knowledge
hospital_burden_ranking = 40 * inpatient_confirmed + 50 * icu_confirmed + 10 * staff_vaccinated

In [5]:
# creating a dataframe "metric" that has all the required columns to create the new features
metric = master_data[["staffed_icu_adult_patients_confirmed_covid_7_day_avg", 
              "staffed_icu_pediatric_patients_confirmed_covid_7_day_avg",
              "total_icu_beds_7_day_avg",
              "inpatient_beds_used_covid_7_day_avg", 
              "inpatient_beds_7_day_avg",
              "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg", 
              "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg", 
              "total_personnel_covid_vaccinated_doses_none_7_day",
              "total_personnel_covid_vaccinated_doses_one_7_day",
              "total_personnel_covid_vaccinated_doses_all_7_day"]]

In [6]:
metric['inpatient_confirmed'] = metric.apply(lambda row: row.inpatient_beds_used_covid_7_day_avg / row.inpatient_beds_7_day_avg, axis=1)

metric['icu_confirmed'] = metric.apply(lambda row: (row.staffed_icu_adult_patients_confirmed_covid_7_day_avg + row.staffed_icu_pediatric_patients_confirmed_covid_7_day_avg)/ row.total_icu_beds_7_day_avg, axis=1) 

metric['staff_vaccinated'] = metric.apply(lambda row: row.total_personnel_covid_vaccinated_doses_all_7_day/( row.total_personnel_covid_vaccinated_doses_none_7_day + row.total_personnel_covid_vaccinated_doses_one_7_day + row.total_personnel_covid_vaccinated_doses_all_7_day), axis=1)


/var/folders/3r/253j8tf90m3_kn7g7q57br8r0000gp/T/ipykernel_26024/586124474.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  metric['inpatient_confirmed'] = metric.apply(lambda row: row.inpatient_beds_used_covid_7_day_avg / row.inpatient_beds_7_day_avg, axis=1)
/var/folders/3r/253j8tf90m3_kn7g7q57br8r0000gp/T/ipykernel_26024/586124474.py:1: RuntimeWarning: invalid value encountered in double_scalars
  metric['inpatient_confirmed'] = metric.apply(lambda row: row.inpatient_beds_used_covid_7_day_avg / row.inpatient_beds_7_day_avg, axis=1)
/var/folders/3r/253j8tf90m3_kn7g7q57br8r0000gp/T/ipykernel_26024/586124474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metric['inpatient_confirmed'] = metric.apply(lambda row: row.inpatie

In [7]:
metric_col = metric[["inpatient_confirmed", "icu_confirmed", "staff_vaccinated"]]

In [8]:
metric_col.describe()

,inpatient_confirmed,icu_confirmed,staff_vaccinated
count,7.308520e+05,4.876410e+05,204416.000000
mean,inf,inf,0.556532
std,NaN,NaN,0.243781
min,0.000000e+00,0.000000e+00,0.000000
25%,2.380952e-03,0.000000e+00,0.389313
50%,3.469388e-02,3.467406e-02,0.494679
75%,9.047619e-02,1.400000e-01,0.759830
max,inf,inf,1.000000


In [11]:
# dealing with the "inf" and replacing it with the finite max-value 
metric_col['inpatient_confirmed'].replace([np.inf], 59.642857, inplace=True)
metric_col['icu_confirmed'].replace([np.inf], 7, inplace=True)

/var/folders/3r/253j8tf90m3_kn7g7q57br8r0000gp/T/ipykernel_26024/4049107945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metric_col['inpatient_confirmed'].replace([np.inf], 59.642857, inplace=True)
/var/folders/3r/253j8tf90m3_kn7g7q57br8r0000gp/T/ipykernel_26024/4049107945.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metric_col['icu_confirmed'].replace([np.inf], 7, inplace=True)


In [12]:
metric_col.describe()

,inpatient_confirmed,icu_confirmed,staff_vaccinated
count,730852.000000,487641.000000,204416.000000
mean,1.159295,0.109623,0.556532
std,7.996425,0.247478,0.243781
min,0.000000,0.000000,0.000000
25%,0.002381,0.000000,0.389313
50%,0.034694,0.034674,0.494679
75%,0.090476,0.140000,0.759830
max,59.642857,7.000000,1.000000


In [14]:
#min-max scaling the data

# creating a copy
df_sklearn = metric_col.copy()
  
# apply normalization techniques
column = 'inpatient_confirmed'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))
  
# view normalized data  
#display(df_sklearn)

In [15]:
column = 'icu_confirmed'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))

In [16]:
column = 'staff_vaccinated'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))

In [17]:
# replacing the null values with 0
df_sklearn = df_sklearn.fillna(0)


In [18]:
df_sklearn.describe()

,inpatient_confirmed,icu_confirmed,staff_vaccinated
count,742253.000000,742253.000000,742253.000000
mean,0.019139,0.010289,0.153269
std,0.133060,0.029604,0.279597
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000565,0.000000,0.000000
75%,0.001490,0.010204,0.280374
max,1.000000,1.000000,1.000000


In [19]:
df_sklearn['hbr'] = df_sklearn.apply(lambda row: 40 * row.inpatient_confirmed + 50 * row.icu_confirmed + 10 * row.staff_vaccinated, axis=1)


In [20]:
df_sklearn.describe()

,inpatient_confirmed,icu_confirmed,staff_vaccinated,hbr
count,742253.000000,742253.000000,742253.000000,742253.000000
mean,0.019139,0.010289,0.153269,2.812660
std,0.133060,0.029604,0.279597,6.016567
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.013405
50%,0.000565,0.000000,0.000000,0.370409
75%,0.001490,0.010204,0.280374,4.063838
max,1.000000,1.000000,1.000000,97.329192


# Further Steps:
This gives us ranking of Hospital Burden based on the initialized weights (that were chosen considering various factors and domain knowledge) 

To update the weights of the equation calculating HBR, we came up with an algortihm (which could not be implemented due to the time constraint)

- Create 10 clusters using these new developed features (inpatient_confirmed, icu_confirmed, staff_vaccinated) using K-means clustering. 
- These clusters are representative of "unordered" Hospital Burden (Basically, each of this cluster represents one bin of the ranked Hospital burden. Let's say 80-90 ranked could be closely represented by cluster 5, and similarly other bins of ranks could be represented by some other cluster.) 
- So, we try to map the cluster to these bins and calculate the weights each of these features carry using grid-search in a way that we have 10 clusters closesly represent the 10 bins of HBR. 

Due to the time constraint, we were not able to complete this algo. 

